In [1]:
import sqlite3
import PIL
import io
import requests
import sys
import config
import tweepy
import nltk
import re
import time
import os
import streamlit as st
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

from streamlit import caching
from PIL import Image
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from textblob import TextBlob
from twitter_stream import MaxStreamer, TwitterClient, TweetAnalyzer

In [2]:
consumer_key = config.CONSUMER_KEY
consumer_secret = config.CONSUMER_SECRET
access_token = config.ACCESS_TOKEN
access_token_secret = config.ACCESS_TOKEN_SECRET

In [3]:
api = TwitterClient().get_twitter_client_api()
tweets = api.user_timeline(screen_name="ForexLive", count=100)
tweet_analyzer = TweetAnalyzer()
df = tweet_analyzer.tweets_to_dataframe(tweets)
df.head()

,tweet,polarity,subjectivity,word_count,char_count,word_density,id,favorite_count,retweet_count,date,source
0,Atlanta Fed GDP now sees third-quarter growth ...,0.0,0.0,78,69,1.114286,1290315995735883776,1,0,2020-08-03 15:58:00,ForexLive Tweets
1,European shares end the session with strong ag...,0.0,0.0,75,67,1.102941,1290311598112825345,4,0,2020-08-03 15:40:32,ForexLive Tweets
2,Arizona cases up 1030 on the day https://t.co/...,0.0,0.0,56,49,1.120000,1290307206206775296,2,2,2020-08-03 15:23:05,ForexLive Tweets
3,EURUSD bounces after test of cluster of suppor...,0.0,0.0,84,74,1.120000,1290305651550195712,6,0,2020-08-03 15:16:54,ForexLive Tweets
4,NZDUSD cracks below floor/50% retracement and ...,0.0,0.0,79,71,1.097222,1290302567248015360,4,1,2020-08-03 15:04:39,ForexLive Tweets


In [4]:
import requests
from bs4 import BeautifulSoup

def get_traders_names():
    headers = {
        "User-Agent": 
        "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Mobile Safari/537.36"}
    url = "https://www.forexcrunch.com/60-top-forex-twitter-accounts/"
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")
    table = soup.find(name="ol")
    traders = []
    for element in table.find_all(name="li"):
        trader = element.find(name="a").text.replace("@", "")
        traders.append(trader)
    return traders
# traders = get_traders_names()

def get_all_trader_twitter():
    traders = get_traders_names()
    api = TwitterClient().get_twitter_client_api()
    tweet_analyzer = TweetAnalyzer()
    df = pd.DataFrame()
    for trader in tqdm(traders):
        try: 
            tweets = api.user_timeline(screen_name=trader, count=100)
            df_trader = tweet_analyzer.tweets_to_dataframe(tweets)
            df = pd.concat([df, df_trader], axis=0)
        except:
            pass
    return df

# df = get_all_trader_twitter()

In [8]:
def insert_into_db():
    df = get_all_trader_twitter()
    print(df.shape)
    conn = sqlite3.connect('twitter.db')
    cursor = conn.cursor()
    cursor.execute("""DROP TABLE IF EXISTS TEMPTRADER""")
    df.to_sql(name='TEMPTRADER', con=conn, if_exists="replace", index=False)
    cursor.execute(
        '''
        INSERT INTO TRADER
        SELECT A.* FROM TEMPTRADER A
        LEFT JOIN TRADER B 
        ON A.id=B.id 
        WHERE B.id IS NULL
        ''')
    conn.commit()
    conn.close()
    
def read_from_db():
    conn = sqlite3.connect('twitter.db')
    df = pd.read_sql_query("SELECT * FROM TRADER", con=conn)
    return df